In [ ]:
import sys
sys.path.insert(0, './..')

from utils import load_dataset

from openai import OpenAI
client = OpenAI()

In [ ]:
dataset = load_dataset("./../dataset/sqliv1.csv")

In [ ]:
true_positives = 0
false_positives = 0
true_negatives = 0
false_negatives = 0

for query, label in dataset:
    chat_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a professional in web security. \
                    You will be provided with a string. \
                    Determine if this string causes SQL injection. \
                    If the string can cause SQL injection, return 1. If not, return 0.\
                    Return only 1 or 0 for answers. Do not include an explanation."
            },
            {
                "role": "user",
                "content": query
            }
        ]
    )

    if label == "1":
        if chat_completion.choices[0].message.content == "1":
            true_positives += 1
        else:
            false_negatives += 1
            with open('false_negatives.txt', 'a') as f:
                f.write(f'query: {query}\n')
                f.write(f'output: {chat_completion.choices[0].message.content}\n')
                f.write('\n')
    else:
        if chat_completion.choices[0].message.content == "0":
            true_negatives += 1
        else:
            false_positives += 1
            with open('false_positives.txt', 'a') as f:
                f.write(f'query: {query}\n')
                f.write(f'output: {chat_completion.choices[0].message.content}\n')
                f.write('\n')

print("true positives: ", true_positives)
print("false positives: ", false_positives)
print("true negatives: ", true_negatives)
print("false negatives: ", false_negatives)